In [1]:
import sqlite3
import pandas as pd
import joblib
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy 

conn = sqlite3.connect("db/market_place.db")
cursor = conn.cursor()

In [ ]:
# 1. Obtener todos los segmentos únicos
segmentos = pd.read_sql_query("SELECT DISTINCT segment FROM users", conn)

resultados = []

# 2. Entrenar un modelo por segmento
for segmento in segmentos['segment']:
    print(f"\nEntrenando modelo para el segmento: {segmento}")

    # Consultar interacciones usuario-producto
    query = f"""
    SELECT o.user_id, o.product_id, COUNT(*) as interaction
    FROM orders o
    JOIN users u ON o.user_id = u.user_id
    WHERE u.segment = '{segmento}'
    GROUP BY o.user_id, o.product_id;
    """
    df = pd.read_sql_query(query, conn)

    if df.empty or df['user_id'].nunique() < 5:
        print(f"Segmento '{segmento}' con pocos datos. Saltando...")
        continue

    # Optimización: Filtrar usuarios y productos con pocas interacciones
    df_filtered = df.groupby('user_id').filter(lambda x: len(x) > 5)  # Al menos 5 interacciones por usuario
    df_filtered = df_filtered.groupby('product_id').filter(lambda x: len(x) > 5)  # Al menos 5 interacciones por producto

    # Configurar datos para Surprise
    reader = Reader(rating_scale=(1, 20))
    data = Dataset.load_from_df(df[['user_id', 'product_id', 'interaction']], reader)

    # Dividir en entrenamiento y prueba
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    input(trainset)

    # Entrenar modelo
    model = SVD()
    model.fit(trainset)

    # Evaluar modelo
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions, verbose=False)
    print(f"📊 RMSE del modelo del segmento '{segmento}': {rmse:.4f}")

    # # Guardar modelo
    # model_path = f"models/SVD/SVD_{segmento}.joblib"
    # joblib.dump(model, model_path)
    # print(f"✅ Modelo guardado: {model_path}")

    # Guardar resultados
    resultados.append({'segmento': segmento, 'rmse': rmse})

# Mostrar resumen de métricas
resultados_df = pd.DataFrame(resultados)
print("\n📈 Resultados generales por segmento:")
print(resultados_df.sort_values(by="rmse"))


Entrenando modelo para el segmento: Ocasional
📊 RMSE del modelo del segmento 'Ocasional': 2.3891
✅ Modelo guardado: models/SVD/SVD_Ocasional.joblib

Entrenando modelo para el segmento: Inactivo
📊 RMSE del modelo del segmento 'Inactivo': 1.7306
✅ Modelo guardado: models/SVD/SVD_Inactivo.joblib

Entrenando modelo para el segmento: Frecuente
📊 RMSE del modelo del segmento 'Frecuente': 5.8610
✅ Modelo guardado: models/SVD/SVD_Frecuente.joblib

📈 Resultados generales por segmento:
    segmento      rmse
1   Inactivo  1.730642
0  Ocasional  2.389075
2  Frecuente  5.860992
